In [72]:
# Import necessary libraries
import pandas as pd
import numpy as np
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from gensim.models import Word2Vec, KeyedVectors
import matplotlib.pyplot as plt

In [73]:
# Load the data
file_path = 'BusinessNewsData.csv'
data = pd.read_csv(file_path)
data.head() 

,Title,Description,Meta-Summary,Summary,syndicate-catagorys,Keywords-for-related-articles
0,টাকা ফেরতে ই–অরেঞ্জের লিখিত\r\nপ্রতিশ্রুতি চান...,ই–অরেঞ্জের মালিকপক্ষ যেন বাণিজ্য মন্ত্রণালয়ের ...,ভুক্তভোগী গ্রাহকদের পক্ষে চারজন আজ বুধবার সচিব...,ভুক্তভোগী গ্রাহকদের পক্ষে চারজন আজ বুধবার সচিব...,বাণিজ্য,"ই কমার্স, বাণিজ্য"
1,ক্লিংকার রপ্তানিতে শুল্ক বৃদ্ধির প্রস্তাব ভিয়ে...,সিমেন্ট তৈরির প্রধান কাঁচামাল ক্লিংকার রপ্তানি...,ভিয়েতনামের শুল্ক বাড়ানো হলে ক্লিংকারের দর আরেক...,ভিয়েতনামের শুল্ক বাড়ানো হলে ক্লিংকারের দর আরেক...,বাণিজ্য,"আমদানি রপ্তানি, সিমেন্ট খাত"
2,সূচক ও লেনদেন কমেছে শেয়ারবাজারে,সূচক ও লেনদেন কমেছে দেশের শেয়ারবাজারে। আজ বুধব...,সূচক ও লেনদেন কমেছে দেশের শেয়ারবাজারে। আজ বুধব...,সূচক ও লেনদেন কমেছে দেশের শেয়ারবাজারে। আজ বুধব...,বাণিজ্য,"ডিএসই, সিএসই, ডিএসইএক্স"
3,এমডির চলতি দায়িত্বে শিরীন আখতার,বাংলাদেশ কৃষি ব্যাংকের উপব্যবস্থাপনা পরিচালক শ...,বাংলাদেশ কৃষি ব্যাংকের উপব্যবস্থাপনা পরিচালক শ...,বাংলাদেশ কৃষি ব্যাংকের উপব্যবস্থাপনা পরিচালক শ...,NaN,ব্যাংক
4,মালা শাড়ির আনোয়ার হোসেন,"মালা শাড়ি, বাংলাদেশের শাড়ির জগতে প্রথম সুপরিচি...",দেশের অন্যতম বৃহৎ শিল্পগোষ্ঠী আনোয়ার গ্রুপের চ...,দেশের অন্যতম বৃহৎ শিল্পগোষ্ঠী আনোয়ার গ্রুপের চ...,NaN,"উদ্যোক্তা, ব্যবসায়ী, অর্থনীতি"


In [74]:
# Data Cleaning: Remove NaN values
data = data.dropna(subset=['Description', 'Keywords-for-related-articles'])

In [75]:
from bnlp import BengaliCorpus as corpus

# Define Bangla stopwords
bangla_stopwords=set(corpus.stopwords)

# Function for preprocessing text
def preprocess_bangla_text(text):
    # Normalize whitespace
    text = ' '.join(text.split())
    
    # Remove stopwords
    text = ''.join(word for word in text.split() if word not in bangla_stopwords)
    
    # Define Bangla punctuation except full stop (।)
    bangla_punctuation = r'[!\"#$%&\'()*+,-./:;<=>@\[\\\]^_`{|}~]–?'
    
    # Remove all Bangla punctuation except full stop (।)
    text = re.sub(bangla_punctuation, '', text)
    
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    
    return text

In [76]:
# Apply preprocessing to the 'Description' column
data['preprocess_description'] = data['Description'].apply(preprocess_bangla_text)
print(data['preprocess_description'].head())

0    ই–অরেঞ্জের মালিকপক্ষ বাণিজ্য মন্ত্রণালয়ের কর্ম...
1    সিমেন্ট তৈরির প্রধান কাঁচামাল ক্লিংকার রপ্তানি...
2    সূচক লেনদেন কমেছে দেশের শেয়ারবাজারে। বুধবার চল...
3    বাংলাদেশ কৃষি ব্যাংকের উপব্যবস্থাপনা পরিচালক শ...
4    মালা শাড়ি বাংলাদেশের শাড়ির জগতে সুপরিচিত ব্র্য...
Name: preprocess_description, dtype: object


In [77]:
# Function for preprocessing text
from bnlp import CleanText

clean_text = CleanText(
   fix_unicode=True,
   unicode_norm=True,
   unicode_norm_form="NFKC",
   remove_url=True,
   remove_email=True,
   remove_emoji=True,
   remove_number=False,
   remove_digits=False,
   remove_punct=False,
   replace_with_url="",
   replace_with_email="<EMAIL>",
   replace_with_number="<NUMBER>",
   replace_with_digit="<DIGIT>",
   replace_with_punct = ""
)

data['clean_description'] = data['preprocess_description'].apply(clean_text)
print("Clean descriptions:")
print(data['clean_description'].head(20))

Clean descriptions:
0     ই–অরেঞ্জের মালিকপক্ষ বাণিজ্য মন্ত্রণালয়ের কর্...
1     সিমেন্ট তৈরির প্রধান কাঁচামাল ক্লিংকার রপ্তানি...
2     সূচক লেনদেন কমেছে দেশের শেয়ারবাজারে। বুধবার চ...
3     বাংলাদেশ কৃষি ব্যাংকের উপব্যবস্থাপনা পরিচালক শ...
4     মালা শাড়ি বাংলাদেশের শাড়ির জগতে সুপরিচিত ব্র...
5     তালেবান কাবুলের নিয়ন্ত্রণ করতেই প্রেসিডেন্ট আ...
6     রাজধানীর আগারগাঁও এলাকায় একসময় বস্তি খালি জা...
7     রাজশাহীর গোদাগাড়ীতে ২০১৮ সালে দেড় শ টাকা খরচ...
8     ইকমার্স প্রতিষ্ঠান ই–অরেঞ্জের মালিকপক্ষ প্রতার...
9     পর্যটন খাতের ব্যবসায়ীরা টিকে থাকতে চান। গ্রাহ...
10    সময়মতো ঋণের কিস্তি পরিশোধ নারী উদ্যোক্তারা ১ ...
11    দেশের অন্যতম বৃহৎ শিল্পগোষ্ঠী আনোয়ার গ্রুপের ...
12    গত রোববার তালেবান আফগানিস্তানের রাজধানী কাবুলে...
13    'ডিজিটাল মুদ্রা' শব্দটি শুনলে মানুষের চোখের সা...
14    সপ্তাহ হওয়া তালেবানের আফগানিস্তান নিয়ন্ত্রণে...
15    আফগানিস্তানের ভঙ্গুর অর্থনীতি এগিয়ে চলেছে মূল...
16    ৬৩ ধরনের পণ্য পচনশীল হিসেবে গণ্য হবে। জাতীয় র...
17    ক্ষুদ্র কৃষক উন্নয়ন ফ

In [78]:
# Tokenize the cleaned descriptions
from bnlp import BasicTokenizer

tokenizer = BasicTokenizer()
data['tokens'] = data['clean_description'].apply(tokenizer.tokenize)
print(data['tokens'].head(10))

0    [ই, –, অরেঞ্জের, মালিকপক্ষ, বাণিজ্য, মন্ত্রণাল...
1    [সিমেন্ট, তৈরির, প্রধান, কাঁচামাল, ক্লিংকার, র...
2    [সূচক, লেনদেন, কমেছে, দেশের, শেয়ারবাজারে, ।, ...
3    [বাংলাদেশ, কৃষি, ব্যাংকের, উপব্যবস্থাপনা, পরিচ...
4    [মালা, শাড়ি, বাংলাদেশের, শাড়ির, জগতে, সুপরিচ...
5    [তালেবান, কাবুলের, নিয়ন্ত্রণ, করতেই, প্রেসিডে...
6    [রাজধানীর, আগারগাঁও, এলাকায়, একসময়, বস্তি, খ...
7    [রাজশাহীর, গোদাগাড়ীতে, ২০১৮, সালে, দেড়, শ, ট...
8    [ইকমার্স, প্রতিষ্ঠান, ই, –, অরেঞ্জের, মালিকপক্...
9    [পর্যটন, খাতের, ব্যবসায়ীরা, টিকে, থাকতে, চান,...
Name: tokens, dtype: object


In [80]:
# Stemming Bangla words 
from bangla_stemmer.stemmer import stemmer

# Initialize BasicTokenizer and BanglaStemmer
tokenizer = BasicTokenizer()
stemmer = stemmer.BanglaStemmer()

# Tokenize and stem the cleaned descriptions
def tokenize_and_stem(text):
    tokens = tokenizer.tokenize(text)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

data['stemmed_description'] = data['extracted_sentences'].apply(tokenize_and_stem)

# Display the stemmed descriptions
print("Stemmed descriptions:")
print(data['stemmed_descriptio'].head(10))

applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied third rules..
applied first rules..
applied fourth rules..

applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
ap

applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
a

applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules.

applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied third rules..
applied third rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
ap

...

applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
app

applied fourth rules..
applied fourth rules..
applied second rules..
applied second rules..
applied third rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
a

applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied third rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied third rules..
applied fourth rules..
applied second rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
ap

applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied second rules..
applied second rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..

applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules.

In [92]:
# Initialize NERTagger
from bnlp import BengaliNER

bn_ner = BengaliNER()

def apply_ner(text):
    ner_tags = bn_ner.tag(text)
    return ner_tags

data['ner_tags'] = data['stemmed_description'].apply(apply_ner)

# Display the NER tags
print("NER tags for the stemmed descriptions:")
print(data[['ner_tags']].head(10))

NER tags for the stemmed descriptions:
                                            ner_tags
0  [(–, O), (অরেঞ্জ, O), (মালিকপক্ষ, O), (বাণিজ্য...
1  [(সিমেন্ট, O), (তৈরির, O), (প্রধান, O), (কাঁচা...
2  [(সূচক, O), (লেনদ, O), (কম, O), (দেশ, O), (শেয...
3  [(বাংলাদেশ, S-LOC), (কৃষি, B-ORG), (ব্যাংক, E-...
4  [(মালা, S-LOC), (শাড়ি, S-LOC), (বাংলাদেশ, S-L...
5  [(তালেবান, O), (কাবুল, O), (নিয়ন্ত্রণ, O), (প...
6  [(রাজধানীর, O), (আগারগাঁ, O), (এলাকায়, O), (এ...
7  [(রাজশাহীর, S-LOC), (গোদাগাড়ী, S-LOC), (২০১৮,...
8  [(ইকমার্স, O), (প্রতিষ্ঠান, O), (–, O), (অরেঞ্...
9  [(পর্যটন, O), (খাত, O), (ব্যবসায়ী, O), (টিক, ...


In [97]:
import pandas as pd
from bnlp import BengaliNER
import re

# Initialize NERTagger
bn_ner = BengaliNER()


def clean_ner(text):
    ner_tags = bn_ner.tag(text)
    
    # Function to tag numbers as DATE
    def tag_numbers(ner_tags):
        tagged_entities = []
        for word, tag in ner_tags:
            if re.match(r'\d+', word):  # Simple regex to match numbers
                tagged_entities.append((word, 'NUMBER'))
            else:
                tagged_entities.append((word, tag))
        return tagged_entities
    
    # Apply post-processing
    processed_ner_tags = tag_numbers(ner_tags)
    
    # Extract named entities including manually tagged numbers
    named_entities = [word for word, tag in processed_ner_tags if tag != 'O']
    return ' '.join(named_entities)

data['ner_named_entities'] = data['stemmed_description'].apply(clean_ner)

# Display the named entities
print(data['ner_named_entities'].head())

0    কর্মকর্তা মালিকপক্ষ টাকা নড়চড় সাগর মন্ত্রণাল...
1                                                    ৫
2                          ঢাকা স্টক এক্সচেঞ্জ ১৫ ৬৭৭১
3                            বাংলাদেশ কৃষি ব্যাংক ১৯৮৮
4    মালা শাড়ি বাংলাদেশ শাড়ি শাড়ি মালা শাড়ি মাল...
Name: ner_named_entities, dtype: object


In [124]:
# Extract unique named entities from all articles
all_named_entities = set()
for entities in data['ner_named_entities']:
    all_named_entities.update(entities.split())

# Function to find sentences with named entities
def find_related_sentences(description, named_entities):
    sentences = description.split('.')
    related_sentences = [sentence for sentence in sentences if any(entity in sentence for entity in named_entities)]
    return '. '.join(related_sentences)

# Apply the function to stemmed descriptions
data['related_sentences'] = data['stemmed_description'].apply(find_related_sentences, named_entities=all_named_entities)

# Display the related sentences
print(data['related_sentences'].head())

0    – অরেঞ্জ মালিকপক্ষ বাণিজ্য মন্ত্রণালয় কর্মকর্...
1    সিমেন্ট তৈরির প্রধান কাঁচামাল ক্লিংক রপ্তানি শ...
2    সূচক লেনদ কম দেশ শেয়ারবাজার । বুধব চলতি সপ্তা...
3    বাংলাদেশ কৃষি ব্যাংক উপব্যবস্থাপনা পরিচালক শির...
4    মালা শাড়ি বাংলাদেশ শাড়ির জগ সুপরিচিত ব্র্যান...
Name: related_sentences, dtype: object


In [133]:
from collections import defaultdict

# Initialize NERTagger
bn_ner = BengaliNER()

# ner tag for finding numbers
def tag_numbers(ner_tags):
    tagged_entities = []
    bangla_digit_pattern = r'[০-৯]+'
    bangla_date_pattern = r'[০-৯]{1,2}/[০-৯]{1,2}/[০-৯]{2,4}'
    for word, tag in ner_tags:
        if re.match(bangla_digit_pattern, word) or re.match(bangla_date_pattern, word):
            tagged_entities.append((word, 'NUMBER'))
        else:
            tagged_entities.append((word, tag))
    return tagged_entities

def extract_entities(ner_tags):
    entities = []
    current_entity = []
    current_tag = None
    
    for word, tag in ner_tags:
        if tag != 'O':
            if tag == current_tag:
                current_entity.append(word)
            else:
                if current_entity:
                    entities.append((' '.join(current_entity), current_tag))
                current_entity = [word]
                current_tag = tag
        else:
            if current_entity:
                entities.append((' '.join(current_entity), current_tag))
                current_entity = []
                current_tag = None
    
    if current_entity:
        entities.append((' '.join(current_entity), current_tag))
    
    return entities

def clean_ner(text):
    ner_tags = bn_ner.tag(text)
    processed_ner_tags = tag_numbers(ner_tags)
    entities = extract_entities(processed_ner_tags)
    return ' '.join([entity for entity, _ in entities])

def create_entity_index(data):
    entity_index = defaultdict(set)
    for idx, row in data.iterrows():
        entities = row['ner_named_entities'].split()
        for entity in entities:
            entity_index[entity].add(idx)
    return entity_index

def expand_query(query):
    query_ner = bn_ner.tag(query)
    expanded_query = []
    for word, tag in query_ner:
        expanded_query.append(word)
        if tag != 'O':
            expanded_query.append(word)
    return ' '.join(expanded_query)

In [ ]:
def calculate_relevance_score(query_entities, document_entities):
    query_set = set(query_entities.split())
    doc_set = set(document_entities.split())
    
    bangla_digit_pattern = r'[০-৯]+'
    bangla_date_pattern = r'[০-৯]{1,2}/[০-৯]{1,2}/[০-৯]{2,4}'
    
    weighted_score = 0
    for i, entity in enumerate(document_entities.split()):
        if entity in query_set:
            if re.match(bangla_digit_pattern, entity) or re.match(bangla_date_pattern, entity):
                weighted_score += 2 / (i + 1)  # Higher weight for numbers
            else:
                weighted_score += 1 / (i + 1)  # Normal weight for other entities
    
    return weighted_score / len(query_set) if query_set else 0



In [ ]:
# summary based on entity
def entity_based_summary(text, named_entities, max_length=1000):
    words = text.split()
    entities = named_entities.split()
    bangla_digit_pattern = r'[০-৯]+'
    bangla_date_pattern = r'[০-৯]{1,2}/[০-৯]{1,2}/[০-৯]{2,4}'
    number_entities = [e for e in entities if re.match(bangla_digit_pattern, e) or re.match(bangla_date_pattern, e)]
    other_entities = [e for e in entities if e not in number_entities]
    
    summary = []
    context_window = 2  # Number of words to include before and after each entity
    
    def add_with_context(word_index):
        start = max(0, word_index - context_window)
        end = min(len(words), word_index + context_window + 1)
        summary.extend(words[start:end])
    
    # Add numbers with context first
    for i, word in enumerate(words):
        if word in number_entities and len(' '.join(summary)) < max_length:
            add_with_context(i)
    
    # Then add other entities with context
    for i, word in enumerate(words):
        if word in other_entities and len(' '.join(summary)) < max_length and word not in summary:
            add_with_context(i)
        if len(' '.join(summary)) >= max_length:
            break
    
    return ' '.join(summary)

In [136]:
def retrieve_information(query, data, entity_index):
    expanded_query = expand_query(query)
    query_entities = clean_ner(expanded_query)
    
    # Calculate relevance scores
    scores = []
    for idx, row in data.iterrows():
        score = calculate_relevance_score(query_entities, row['ner_named_entities'])
        scores.append((idx, score))
    
    # Sort by relevance score
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
    
    # Get top 5 results
    top_results = sorted_scores[:5]
    
    # Generate summaries for top results
    results = []
    for idx, score in top_results:
        summary = entity_based_summary(data.loc[idx, 'stemmed_description'], data.loc[idx, 'ner_named_entities'])
        results.append({
            'index': idx,
            'score': score,
            'summary': summary,
            'Stemmed Text': data.loc[idx, 'stemmed_description']
        })
    
    return results

# Apply NER to the data
data['ner_named_entities'] = data['stemmed_description'].apply(clean_ner)

# Create entity index
entity_index = create_entity_index(data)

# Example usage
query = "শেয়ারবাজার"
results = retrieve_information(query, data, entity_index)
print("Query:", query)
print("\nTop 5 Results:")
for result in results:
    print(f"Index: {result['index']}")
    print(f"Relevance Score: {result['score']}")
    print(f"Summary: {result['summary']}")
    print(f"Stemmed Text:{result['Stemmed Text']}")
    print()

results_df = pd.DataFrame(results)

Query: শেয়ারবাজার

Top 5 Results:
Index: 0
Relevance Score: 0
Summary: মালিকপক্ষ প্রতারণামূলকভাব ১ ১০০ টাকা প্রতারণামূলকভাব ১ ১০০ টাকা আত্মসাৎ মালিকপক্ষ বাণিজ্য মন্ত্রণালয় কর্মকর্তা গ্রাহক ভবিষ্য কোনোভাব নড়চড় । সাগর বাণিজ্য মন্ত্রণালয় ডব্লিউ সেল মহাপরিচালক সেল মহাপরিচালক মো হাফিজুর রহমান
Stemmed Text:– অরেঞ্জ মালিকপক্ষ বাণিজ্য মন্ত্রণালয় কর্মকর্তা গ্রাহক পণ্য টাকা ফেরত রিফান্ড দেওয় লিখিত প্রতিশ্রুতি দা জানিয় – অরেঞ্জ ভুক্তভোগী গ্রাহকে । বুধব সচিবালয় বাণিজ্য মন্ত্রণালয় অতিরিক্ত সচিব আবেদনপত্র জমা । আবেদন পণ্য টাকা ফেরত পেত বাণিজ্য মন্ত্রণালয় হস্তক্ষেপ কামনা । আবেদন – অরেঞ্জ অফিশিয়াল ফেসবুক পাজ পণ্য টাকা ফেরত দেওয় প্রতিশ্রুতি দে বারব । বারব ভঙ্গ কর । প্রতিশ্রুতি ভুক্তভোগী গ্রাহক সামনাসামনি । মালিকপক্ষ গ্রাহক টাকা কীভাব ফেরত দাব ফেরত দাব সুনির্দিষ্টভাব । পণ্য টাকা ফেরত আলোচন সময় সামা ভাউচ স্বপ্ন ভাউচ টাকা ফেরত লিখিত প্রতিশ্রুতি । – অরেঞ্জ মালিক পক্ষ লিখিত স্ট্যাম্প ভবিষ্য কোনোভাব নড়চড় । সাগর সরক নাম এক গ্রাহক নেতৃত্ব চারজন গ্রাহক বাণিজ্য মন্ত্রণালয় ডব্লিউ সেল মহাপরিচালক ম